In [411]:
import allel
from cruzdb import Genome
import os
from tqdm import tqdm_notebook as tqdm
# import argparse
# from shutil import copyfile
import pandas as pd

In [405]:
# BAM="/media/emir/Seagate/data/patients"
# path_to_output="/media/emir/Storage/Cancer/mutect/rvboost"
# usb="/media/emir/USB/RVBoost DV primary files/"

# for i in range(1,71):
#     try:
#         copyfile(os.path.join(usb,"%d.rvboost.s1s2.filtered.tsv" % (i)), 
#                  os.path.join(path_to_output,"%d_s1s2" % (i),"%d.rvboost.s1s2.filtered.tsv" % (i)))
#     except IOError:
#         print "skldf"
#     s=i.split("_")[1]+"_s1s2"
#     s2=i.split("_")[1]+"s2"
#     if i.split("_")[1] not in [1,2,3,4,"T"]:
#         os.mkdir(os.path.join(path_to_output, s))

<center><h4> Добавляем к себе базы данных refGene и knownGene </h4></center>

In [373]:
path_to_db_sqlite = "sqlite:////media/emir/Storage/Cancer/mutect/tb_db" 
# g=Genome("hg19").mirror(["refGene", "knownGene", "kgXref"],
#                         os.path.join(path_to_db_sqlite, "hg19.db"))
# g=Genome("hg38").mirror(["refGene", "knownGene", "kgXref"],
#                         os.path.join(path_to_db_sqlite, "hg38.db"))

In [265]:
g_19=Genome(os.path.join(path_to_db_sqlite, "hg19.db"))
# g_38=Genome(os.path.join(path_to_db_sqlite, "hg38.db"))

<center><h4> Делаем из файла запускаемый скрипт </center></h4>

In [4]:
# def main():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--patnum", "-pn", help="patient number", action="store")
#     parser.add_arguments("--sample", help="number of sample, e.g. 1 or 2", 
#                          action="store")
#     args = parser.parse_args()
#     print make_gl(args.patnum, args.sample)

<center><h4>Создаем bed-файл с уникальным списком генов для каждой пары образцов</h4></center>

1) Вначале пробовал брать названия генов типа "TP53", но слишком много генов не находилось (из 8300 генов не было найдено около 1000). Решил взять идентификаторы транскриптов. Выход сильно больше (потерей около 300 транскриптов). <br>
2) А потом понял, что можно искать вначале по идентификатору транскрипта (вроде он уникален -- но все равно проверим), а потом, если вдруг genome browser его не нашел, искать по названию гена. <br>
3) Важно учитывать альтернативный сплайсинг -- один ген имеет несколько транскриптов. Для простоты брал самый длинный.

In [466]:
def make_gl_5pat(patient, sample):
    
    if isinstance(patient, str):
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ss%d" % (patient, sample)
        file_name = "intersect.%s_s%d_0.0.vcf" % (patient, sample)
    else:
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ds%d" % (patient, sample)
        file_name = "intersect.%d_s%d_0.0.vcf" % (patient, sample)
    callset=allel.vcf_to_dataframe(os.path.join(path_to_files, file_name),
                                   fields=["SNPEFF_TRANSCRIPT_ID","SNPEFF_GENE_NAME"], alt_number=1)
    callset.fillna("-", inplace=True) 
    callset.query("SNPEFF_TRANSCRIPT_ID != \"-\" | SNPEFF_GENE_NAME != \"-\"",inplace=True)
    callset.sort_values("SNPEFF_GENE_NAME", inplace=True)
    callset.drop_duplicates(inplace=True)
                  
#     with open(os.path.join(path_to_files, "genes_pat%ds%d_new.txt" % (patient, sample)),
#               "w") as genes_str:
#         for i in tqdm(gene_list):
#             genes_str.write(i + "\n")

    if isinstance(patient, str):
        output_name="genes_pat%ss%d_refGene_new.bed" % (patient, sample)
    else:
        output_name="genes_pat%ds%d_refGene_new.bed" % (patient, sample)

        
    with open(os.path.join(path_to_files, output_name),
              "w") as genes_str:
        """
        Проверяем, есть ли transcript id в genome browser
        """
        for i in tqdm(range(callset.shape[0])):
            transcr=callset.iloc[i,0]
            gene_name = g_19.refGene.filter_by(name=transcr).first()
            if gene_name is None:
                """
                Если transcript id нет, значит смотрим на gene name
                """
                gene=callset.iloc[i,1]
                
                """
                Этот блок нужен для того, чтобы не искать один и тот же ген дважды
                """
                
                if (i!=0) and (callset.iloc[i-1,1]==callset.iloc[i,1]):
                    if (callset.iloc[i-1,1]=="-") and (callset.iloc[i-1,1]=="-"):
                        if (callset.iloc[i-1,0]==callset.iloc[i,0]):
                            continue
                    else:
                        continue
                
                """
                Причем запариваемся с прочерками
                Если встречаем две черточки подряд, то смотрим на transcript id
                Если одинаквый, то continue, если нет, идем по циклу
                UPD: вообще, вроде черточек в генах не видел, но блок пускай будет
                """
                
                d={}
                gene_name = g_19.refGene.filter_by(name2=gene).all()
                if not len(gene_name):
                    continue
                for i in gene_name:
                    # Приходится так делать, потому что 
                    # не знаю как по-другому работать с refgene объектом
                    i=str(i); i=i.split("\t")
                    d[i[1],i[2]]=int(i[2])-int(i[1])
                a=max(d)
                for i,j in enumerate(gene_name):
                    j=str(j); j=j.split("\t")
                    if a[0] == j[1] and a[1] == j[2]:
                        genes_str.write(gene_name[i].bed() + "\n")
                        break
            else:
                genes_str.write(gene_name.bed() + "\n")
            
    return None

In [469]:
def make(patient, sample):
    
    if isinstance(patient, str):
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ss%d" % (patient, sample)
        file_name = "intersect.%s_s%d_0.0.vcf" % (patient, sample)
    else:
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ds%d" % (patient, sample)
        file_name = "intersect.%d_s%d_0.0.vcf" % (patient, sample)
    callset=allel.vcf_to_dataframe(os.path.join(path_to_files, file_name),
                                   fields="*", alt_number=1)
    return callset.columns

In [450]:
def make_gl_other_pats(patient):
    
    if isinstance(patient, str):
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%s_s1s2" % (patient)
        file_name = "%s.rvboost.s1s2.filtered.tsv" % (patient)
    else:
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%d_s1s2" % (patient)
        file_name = "%d.rvboost.s1s2.filtered.tsv" % (patient)
    callset=pd.read_csv(os.path.join(path_to_files, file_name), sep="\t", 
                       usecols=["GENE","EFFECTS_DETAILED"])
    callset["split"] = callset["EFFECTS_DETAILED"].str.split("|")
#     callset.fillna("-", inplace=True) 
#     callset.query("SNPEFF_TRANSCRIPT_ID != \"-\" | SNPEFF_GENE_NAME != \"-\"",inplace=True)
#     callset.sort_values("SNPEFF_GENE_NAME", inplace=True)
#     callset.drop_duplicates(inplace=True)
                  
#     with open(os.path.join(path_to_files, "genes_pat%ds%d_new.txt" % (patient, sample)),
#               "w") as genes_str:
#         for i in tqdm(gene_list):
#             genes_str.write(i + "\n")

#     if isinstance(patient, str):
#         output_name="genes_pat%ss%d_refGene_new.bed" % (patient, sample)
#     else:
#         output_name="genes_pat%ds%d_refGene_new.bed" % (patient, sample)

        
#     with open(os.path.join(path_to_files, output_name),
#               "w") as genes_str:
#         """
#         Проверяем, есть ли transcript id в genome browser
#         """
#         for i in tqdm(range(callset.shape[0])):
#             transcr=callset.iloc[i,0]
#             gene_name = g_19.refGene.filter_by(name=transcr).first()
#             if gene_name is None:
#                 """
#                 Если transcript id нет, значит смотрим на gene name
#                 """
#                 gene=callset.iloc[i,1]
                
#                 """
#                 Этот блок нужен для того, чтобы не искать один и тот же ген дважды
#                 """
                
#                 if (i!=0) and (callset.iloc[i-1,1]==callset.iloc[i,1]):
#                     if (callset.iloc[i-1,1]=="-") and (callset.iloc[i-1,1]=="-"):
#                         if (callset.iloc[i-1,0]==callset.iloc[i,0]):
#                             continue
#                     else:
#                         continue
                
#                 """
#                 Причем запариваемся с прочерками
#                 Если встречаем две черточки подряд, то смотрим на transcript id
#                 Если одинаквый, то continue, если нет, идем по циклу
#                 UPD: вообще, вроде черточек в генах не видел, но блок пускай будет
#                 """
                
#                 d={}
#                 gene_name = g_19.refGene.filter_by(name2=gene).all()
#                 if not len(gene_name):
#                     continue
#                 for i in gene_name:
#                     # Приходится так делать, потому что 
#                     # не знаю как по-другому работать с refgene объектом
#                     i=str(i); i=i.split("\t")
#                     d[i[1],i[2]]=int(i[2])-int(i[1])
#                 a=max(d)
#                 for i,j in enumerate(gene_name):
#                     j=str(j); j=j.split("\t")
#                     if a[0] == j[1] and a[1] == j[2]:
#                         genes_str.write(gene_name[i].bed() + "\n")
#                         break
#             else:
#                 genes_str.write(gene_name.bed() + "\n")
            
    return callset

In [461]:
make_gl_other_pats(5).iloc[8,2][12]

''

In [332]:
def trial(patient, sample):
    
    if isinstance(patient, str):
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ss%d" % (patient, sample)
        file_name = "intersect.%s_s%d_0.0.vcf" % (patient, sample)
    else:
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ds%d" % (patient, sample)
        file_name = "intersect.%d_s%d_0.0.vcf" % (patient, sample)
    callset=allel.vcf_to_dataframe(os.path.join(path_to_files, file_name),
                                   fields=["SNPEFF_TRANSCRIPT_ID","SNPEFF_GENE_NAME"], alt_number=1)
    callset.fillna("-", inplace=True) 
    callset.query("SNPEFF_TRANSCRIPT_ID != \"-\" | SNPEFF_GENE_NAME != \"-\"",inplace=True)
    callset.sort_values("SNPEFF_GENE_NAME", inplace=True)
    callset.drop_duplicates(inplace=True)
        
#     var="pat"+str(patient)
#     if patient=="T":var=["MAN1C1","DNHD1","GDPGP1","METTL4","TNFSF14"]
#     if patient==2:var=["AMPD2","ZNF485","SUFU","MGA","DSC3","ZNF571","BRD1"]
        
    c1=0
    c2=0
    for i in tqdm(range(callset.shape[0])):
        transcr=callset.iloc[i,0]
        gene_name = g.refGene.filter_by(name=transcr).first()
        if gene_name is None:
            c1+=1
            gene=callset.iloc[i,1]
            gene_name = g.refGene.filter_by(name2=gene).first()
            if gene_name is None:
                c2+=1
                continue
    
#     return callset
    return c1,c2

In [334]:
trial(2,1)

(286, 84)

In [221]:
def check_gl(patient, sample):
    
    if isinstance(patient, str):
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ss%d" % (patient, sample)
        file_name = "intersect.%s_s%d_0.0.vcf" % (patient, sample)
    else:
        path_to_files = "/media/emir/Storage/Cancer/mutect/rvboost/%ds%d" % (patient, sample)
        file_name = "intersect.%d_s%d_0.0.vcf" % (patient, sample)
    callset=allel.vcf_to_dataframe(os.path.join(path_to_files, file_name),
                                   fields=["SNPEFF_TRANSCRIPT_ID","SNPEFF_GENE_NAME"], alt_number=1)
#     callset=allel.vcf_to_dataframe(os.path.join(path_to_files, file_name),
#                                    fields="*", alt_number=1)
    var="pat"+str(patient)
    if patient=="T":
        var=callset.query("SNPEFF_GENE_NAME == \"MAN1C1\" | SNPEFF_GENE_NAME == \"DNHD1\" | SNPEFF_GENE_NAME == \"GDPGP1\" | SNPEFF_GENE_NAME == \"METTL4\" | SNPEFF_GENE_NAME == \"TNFSF14\"")
    if patient==2:
        var=callset.query("SNPEFF_GENE_NAME == \"AMPD2\" | SNPEFF_GENE_NAME == \"ZNF485\" | SNPEFF_GENE_NAME == \"SUFU\" | SNPEFF_GENE_NAME == \"MGA\" | SNPEFF_GENE_NAME == \"DSC3\" | SNPEFF_GENE_NAME == \"ZNF571\" | SNPEFF_GENE_NAME == \"BRD1\"")
    var.drop_duplicates(inplace=True)
            
    d={}
    for i,j in enumerate(var.iterrows()):
        d[j[1][0],j[1][1]]=False
        
    c=0
    for k,v in d.iteritems():
        if k[0] in var["SNPEFF_TRANSCRIPT_ID"].tolist():
            c+=1
            d[k]=True
                
    if c/len(d) == 1: print "All genes"

    for k,v in tqdm(d.iteritems()):
        gene_name = g.refGene.filter_by(name=k[0]).first()
        if gene_name is None:
            print k[0],k[1] 
    
    return None